In [ ]:
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
import torch

base = StableDiffusionXLPipeline.from_single_file(
    "./sd_xl_base_1.0.safetensors", 
    # torch_dtype=torch.float16,
    # variant="fp16",
    use_safetensors=True,
)
# base = StableDiffusionXLPipeline.from_pretrained(
#     "stabilityai/stable-diffusion-xl-base-1.0", 
#     torch_dtype=torch.float16,
#     variant="fp16",
#     use_safetensors=True,
# )

prompt = "masterpiece, best quality, an alien, tentacles, inhuman anatomy, robe, in Goblin Valley, photorealistic, 8k, detailed, dramatic, moody"
neg_prompt = "low res, ugly, bad hands, too many digits, bad teeth, blurry, blurred background"
image = base(prompt=prompt, negative_prompt=neg_prompt).images[0]
image

In [ ]:
refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)

In [ ]:
n_steps = 40
high_noise_frac = 0.8
use_refiner=True

image = base(
    prompt=prompt,
    negative_prompt=neg_prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent" if use_refiner else "pil",
).images[0]
image = refiner(
    prompt=prompt,
    negative_prompt=neg_prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image[None, :],
).images[0]
image